In [ ]:
%pip install langchain
%pip install pinecone-client
%pip install pypdf
%pip install langchain-text-splitters
%pip install langchain_google_genai

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGING_FACE_API = os.getenv('HUGGING_FACE_API')
PINECONE_API_KEY = os.getenv('PINECONE_API')
GOOGLE_API_KEY = os.getenv('GOOGLE_API')

In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("pdfs/")
data = loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(

    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

text_chunks = text_splitter.split_documents(data)

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

[0.05636945, 0.0048285457, -0.0762591, -0.023642512, 0.05329321]

In [5]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "test"

index = pc.Index(index_name, metric="cosine",
                 host="https://test-f6d2zst.svc.gcp-starter.pinecone.io")

In [13]:
from langchain_pinecone import PineconeVectorStore

pvs = PineconeVectorStore(
    pinecone_api_key=PINECONE_API_KEY, embedding=embeddings, index_name=index_name)

docsearch = pvs.from_texts(
    [t for t in text_chunks], embedding=embeddings, index_name=index_name)

PineconeConfigurationError: You haven't specified an Api-Key.